จาก ep ที่แล้ว

# 0. Magic Commands


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# 1. Import Library

In [2]:
!curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [0]:
from fastai import *
from fastai.vision import *
from fastai.metrics import accuracy

# 2. ข้อมูล



In [0]:
path = untar_data(URLs.PETS)
path_images = path/'images'
filenames = get_image_files(path_images)

# 3. เตรียมข้อมูล

In [0]:
batchsize = 64
np.random.seed(0)
regex_pattern = r'/([^/]+)_\d+.jpg$'

databunch = ImageDataBunch.from_name_re(path_images, filenames, 
                                   regex_pattern, ds_tfms=get_transforms(), 
                                   size=224, bs=batchsize).normalize(imagenet_stats)

# 4. สร้างโมเดล


In [10]:
learner = cnn_learner(databunch, models.resnet34, metrics=accuracy)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 53.0MB/s]


# 5. เริ่มต้นเทรนโมเดล


In [11]:
learner.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.936414,0.294695,0.907984,01:44


# 6. เทรนต่อ

unfreeze layer ทุก Layer ให้สามารถเทรนได้ แล้วเทรนต่อทั้งโมเดล

In [0]:
learner.unfreeze()
learner.lr_find()
learner.recorder.plot()

epoch,train_loss,valid_loss,accuracy,time


In [0]:
learner.fit_one_cycle(1, max_lr=slice(1e-6,1e-4))

# 7. สำเร็จแล้ว



# 8. สรุป


เปรียบเทียบ

# Credit 

* https://course.fast.ai/videos/?lesson=11